In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

In [2]:
# 필요한 모듈은 기억해두자.

In [3]:
targetData = pd.read_csv("../../../stdcode6/dataset/cars.csv")

In [4]:
targetData.head()

,age,gender,miles,debt,income,sales
0,28,0,23,0,4099,620
1,26,0,27,0,2677,1792
2,30,1,58,41576,6215,27754
3,26,1,25,43172,7626,28256
4,20,1,17,6979,8071,4438


In [5]:
targetData.dtypes

age       int64
gender    int64
miles     int64
debt      int64
income    int64
sales     int64
dtype: object

In [6]:
columnList = list(targetData.columns)

In [7]:
columnList

['age', 'gender', 'miles', 'debt', 'income', 'sales']

In [8]:
for i in range(0, len(columnList)):
    try:
        targetData[columnList[i]] = targetData[columnList[i]].astype(int)
    except:
        print(e)

In [9]:
targetData.dtypes

age       int32
gender    int32
miles     int32
debt      int32
income    int32
sales     int32
dtype: object

In [10]:
# columnList를 만들어서 필요한 데이터 타입을 자동 변환

In [11]:
corrData = targetData.corr()

In [12]:
corrData

,age,gender,miles,debt,income,sales
age,1.000000,-0.000702,0.232399,0.218896,0.239644,0.352609
gender,-0.000702,1.000000,-0.031355,-0.033181,-0.034317,-0.036350
miles,0.232399,-0.031355,1.000000,0.544791,0.422141,0.636676
debt,0.218896,-0.033181,0.544791,1.000000,0.491790,0.835541
income,0.239644,-0.034317,0.422141,0.491790,1.000000,0.674685
sales,0.352609,-0.036350,0.636676,0.835541,0.674685,1.000000


In [13]:
targetCorr = list(range(3, 8, 1))

In [14]:
targetCorr

[3, 4, 5, 6, 7]

In [15]:
for i in range(0, len(targetCorr)):
    try:
        targetCorr[i] = targetCorr[i] / 10
    except:
        print(e)

In [16]:
targetCorr

[0.3, 0.4, 0.5, 0.6, 0.7]

In [17]:
# /10 해서 원하는 기준 상관계수 형성하기

In [18]:
featuresList = [0 for i in range(len(targetCorr))]

In [19]:
featuresList

[0, 0, 0, 0, 0]

In [20]:
# 배열은 어떤 값이 들어가 있어도 새로 뒤집어쓸 수 있다.
# 0 for i in range(길이)로 0값만 지니는 배열을 생성하고, 여기에 append하거나 대체해버리자.

In [21]:
for i in range(0, len(targetCorr)):
    featuresList[i] = list(corrData[(abs(corrData.sales) > targetCorr[i]) & \
                                    (abs(corrData.sales) != 1)].index)

In [22]:
featuresList

[['age', 'miles', 'debt', 'income'],
 ['miles', 'debt', 'income'],
 ['miles', 'debt', 'income'],
 ['miles', 'debt', 'income'],
 ['debt']]

In [23]:
newFeatures = [0 for i in range(len(featuresList))]

In [24]:
# 마찬가지로 빈 배열 생성

In [25]:
for i in range(0, len(featuresList)):
    newFeatures[i] = targetData[featuresList[i]]

In [26]:
newFeatures

[     age  miles   debt  income
 0     28     23      0    4099
 1     26     27      0    2677
 2     30     58  41576    6215
 3     26     25  43172    7626
 4     20     17   6979    8071
 ..   ...    ...    ...     ...
 958   22     11   8778    9829
 959   19     23   4850    3470
 960   28     28   9312    2720
 961   50     29  51343    8713
 962   47     15   3735    6406
 
 [963 rows x 4 columns],
      miles   debt  income
 0       23      0    4099
 1       27      0    2677
 2       58  41576    6215
 3       25  43172    7626
 4       17   6979    8071
 ..     ...    ...     ...
 958     11   8778    9829
 959     23   4850    3470
 960     28   9312    2720
 961     29  51343    8713
 962     15   3735    6406
 
 [963 rows x 3 columns],
      miles   debt  income
 0       23      0    4099
 1       27      0    2677
 2       58  41576    6215
 3       25  43172    7626
 4       17   6979    8071
 ..     ...    ...     ...
 958     11   8778    9829
 959     23   4850    

In [27]:
# 각 상관 컬럼과 연관된 features데이터들을 newFeatures에 집어넣었다.

In [28]:
# 이제 혼재된 데이터에서 training과 test를 구분해 넣으면 된다.

In [29]:
targetRatio = 0.7

In [30]:
targetIndex = int(targetData.shape[0] * targetRatio)

In [31]:
targetIndex

674

In [32]:
trainingFeatures = [0 for i in range(len(newFeatures))]

In [33]:
testFeatures = [0 for i in range(0, len(newFeatures))]

In [34]:
label = ["sales"]

In [35]:
newLabel = targetData["sales"]

In [36]:
for i in range(0, len(newFeatures)):
    trainingFeatures[i] = newFeatures[i][:targetIndex].reset_index(drop=True, inplace = False).astype(int)
    testFeatures[i] = newFeatures[i][targetIndex:].reset_index(drop=True, inplace = False).astype(int)

In [37]:
trainingLabel = newLabel[:targetIndex].reset_index(drop=True, inplace=False).astype(int)
testLabel = newLabel[targetIndex:].reset_index(drop=True, inplace=False).astype(int)

In [38]:
trainingFeatures

[     age  miles   debt  income
 0     28     23      0    4099
 1     26     27      0    2677
 2     30     58  41576    6215
 3     26     25  43172    7626
 4     20     17   6979    8071
 ..   ...    ...    ...     ...
 669   30     27   9766    5787
 670   25     12   1411    8381
 671   33     25   9569    6598
 672   48     72  55789   10257
 673   30     29   1427    2214
 
 [674 rows x 4 columns],
      miles   debt  income
 0       23      0    4099
 1       27      0    2677
 2       58  41576    6215
 3       25  43172    7626
 4       17   6979    8071
 ..     ...    ...     ...
 669     27   9766    5787
 670     12   1411    8381
 671     25   9569    6598
 672     72  55789   10257
 673     29   1427    2214
 
 [674 rows x 3 columns],
      miles   debt  income
 0       23      0    4099
 1       27      0    2677
 2       58  41576    6215
 3       25  43172    7626
 4       17   6979    8071
 ..     ...    ...     ...
 669     27   9766    5787
 670     12   1411    

In [39]:
trainingFeatures[0]

,age,miles,debt,income
0,28,23,0,4099
1,26,27,0,2677
2,30,58,41576,6215
3,26,25,43172,7626
4,20,17,6979,8071
...,...,...,...,...
669,30,27,9766,5787
670,25,12,1411,8381
671,33,25,9569,6598
672,48,72,55789,10257


In [40]:
trainingLabel

0        620
1       1792
2      27754
3      28256
4       4438
       ...  
669     9196
670     3779
671     8396
672    29138
673     3310
Name: sales, Length: 674, dtype: int32

In [41]:
testFeatures

[     age  miles   debt  income
 0     30     23    773    4557
 1     37     38  41500   11123
 2     20     13   9933    2538
 3     33     23  56544    7079
 4     43     35   7017    5538
 ..   ...    ...    ...     ...
 284   22     11   8778    9829
 285   19     23   4850    3470
 286   28     28   9312    2720
 287   50     29  51343    8713
 288   47     15   3735    6406
 
 [289 rows x 4 columns],
      miles   debt  income
 0       23    773    4557
 1       38  41500   11123
 2       13   9933    2538
 3       23  56544    7079
 4       35   7017    5538
 ..     ...    ...     ...
 284     11   8778    9829
 285     23   4850    3470
 286     28   9312    2720
 287     29  51343    8713
 288     15   3735    6406
 
 [289 rows x 3 columns],
      miles   debt  income
 0       23    773    4557
 1       38  41500   11123
 2       13   9933    2538
 3       23  56544    7079
 4       35   7017    5538
 ..     ...    ...     ...
 284     11   8778    9829
 285     23   4850    

In [42]:
trainingLabel

0        620
1       1792
2      27754
3      28256
4       4438
       ...  
669     9196
670     3779
671     8396
672    29138
673     3310
Name: sales, Length: 674, dtype: int32

In [43]:
testLabel

0       3267
1      22391
2       4927
3      22752
4      12969
       ...  
284     1593
285     4742
286    12771
287    28511
288     6104
Name: sales, Length: 289, dtype: int32

In [44]:
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor

In [45]:
model_method = tree.DecisionTreeRegressor()

In [46]:
model = [0 for i in range(len(trainingFeatures))]

In [47]:
model

[0, 0, 0, 0, 0]

In [48]:
predict = [0 for i in range(len(model))]

In [56]:
for i in range(0, len(model)):
    model[i] = model_method.fit(trainingFeatures[i], trainingLabel)

In [49]:
for i in range(0, len(model)):
    modelAdjust = model_method.fit(trainingFeatures[i], trainingLabel)
    predict[i] = pd.DataFrame(modelAdjust.predict(testFeatures[i]), columns = {"predict"})

In [60]:
# DataFrame 말고 List 형식으로 append 시키라!

DecisionTreeRegressor()

In [58]:
model[0].predict(testFeatures[0])

ValueError: X has 4 features, but DecisionTreeRegressor is expecting 1 features as input.

In [50]:
predict[0]

,predict
0,5277.0
1,28228.0
2,4765.0
3,29096.0
4,6776.0
...,...
284,3437.0
285,3002.0
286,9143.0
287,26148.0


In [51]:
predict[1]

,predict
0,2993.0
1,25239.0
2,1769.0
3,28575.0
4,1285.0
...,...
284,2184.0
285,4936.0
286,6801.0
287,24984.0


In [ ]:
# model이 새로 선언될 때마다, 기존에 호출되었던 모델과 호환이 이뤄지지는 않는다.
# 새로운 예측값을 담을 배열을 신설하고, 여기에 모델 생성 및 예측 시마다 변하는 값을 저장하자.
# for문 하나만 돌려서는 문제가 발생할 수 있다.

In [52]:
mae = [0 for i in range(len(predict))]
mse = [0 for i in range(len(predict))]

In [53]:
for i in range(0, len(mae)):
    mae[i] = mean_absolute_error(testLabel, predict[i])
    mse[i] = mean_squared_error(testLabel, predict[i])

In [54]:
for i in range(0, len(mae)):
    print(mae[i])

2519.8477508650517
3037.823529411765
3093.0484429065746
3039.8512110726642
3860.7506665154006


In [55]:
for i in range(0, len(mse)):
    print(mse[i])

13173316.560553633
20735559.88581315
21568045.6366782
19973435.435986158
29253734.71082089
